# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Args" data-toc-modified-id="Args-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Args</a></div>

In [2]:
"""
addm_mle.py
Author: Gabriela Tavares, gtavares@caltech.edu

Maximum likelihood estimation procedure for the attentional drift-diffusion
model (aDDM), using a grid search over the 3 free parameters of the model. Data
from all subjects is pooled such that a single set of optimal parameters is
estimated (or from a subset of subjects, when provided).

aDDM simulations are generated for the model with maximum estimated likelihood.
"""

import argparse
import numpy as np
import itertools

from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages

from addm import aDDM
from util import (load_data_from_csv, get_empirical_distributions,
                  save_simulations_to_csv, generate_choice_curves,
                  generate_rt_curves, convert_item_values)

# Args

In [3]:
class arguments:
    
    # "--subject-ids", nargs="+", type=str, default=[],
    # help= List of subject ids. If not provided, all
    # existing subjects will be used.
    subject_ids = []
        
    # "--num-threads", type=int, default=9,
    # help= Size of the thread pool.
    num_threads = 3
    
    # "--trials-per-subject", type=int, default=100,
    # help= Number of trials from each subject to be used
    # in the analysis; if smaller than 1, all trials are used.
    trials_per_subject = None  # None to use all trials?
    
    # --simulations-per-condition", type=int, default=800,
    # help= Number of artificial data trials to be
    # generated per trial condition.
    simulations_per_condition = 100
    
    # "--range-d", nargs="+", type=float, default=[0.005, 0.006, 0.007],
    # help= Search range for parameter d.
    range_d = [0.003, 0.006, 0.009]
    
    # "--range-sigma", nargs="+", type=float, default=[0.065, 0.08, 0.095],
    # help= Search range for parameter sigma.
    range_sigma = [0.03, 0.06, 0.09]
    
    # "--range-theta", nargs="+", type=float, default=[0.4, 0.5, 0.6],
    # help= Search range for parameter theta.
    range_theta = [0.3, 0.5, 0.7]
    
    # "--expdata-file-name", type=str, default="expdata.csv",
    # help= Name of experimental data file.
    expdata_file_name = "made_data/expdata.csv"
    
    # "--fixations-file-name", type=str, default="fixations.csv",
    # help= Name of fixations file.
    fixations_file_name = "made_data/fixations.csv"
    
    # "--save-simulations", default=False, action="store_true", 
    # help= Save simulations to CSV.
    save_simulations = True
    
    # "--save-figures", default=False, action="store_true", 
    # help= Save figures comparing choice and RT curves for data and simulations.
    save_figures = True
    
    # "--verbose", default=False, action="store_true",
    # help= Increase output verbosity.
    verbose = True

args = arguments()

In [4]:
data = load_data_from_csv(
    args.expdata_file_name, args.fixations_file_name,
    convertItemValues=None)

In [5]:
# Begin maximum likelihood estimation using odd trials only.
# Get correct subset of trials.
dataTrials = list()
subjectIds = args.subject_ids if args.subject_ids else data.keys()
for subjectId in subjectIds:
    maxNumTrials = len(data[subjectId]) / 2
    numTrials = (args.trials_per_subject
                 if 1 <= args.trials_per_subject <= maxNumTrials
                 else maxNumTrials)
    trialSet = np.random.choice(
        [trialId for trialId in range(len(data[subjectId]))
         if trialId % 2],
        numTrials, replace=False)
    dataTrials.extend([data[subjectId][t] for t in trialSet])